In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
np.set_printoptions(threshold=np.inf, suppress=True)

Loading data

In [2]:
sonar_data = pd.read_csv('dataset/sonar.all-data', header=None)
print(sonar_data.head())
print(sonar_data.shape)
print(sonar_data[60].value_counts())

In [3]:
sonar_data.describe()

In [4]:
# replacnutie R za 0 a M za 1
# R - Rock M - Mina
sonar_data[60] = sonar_data[60].replace(['R', 'M'], [0, 1])
sonar_data.head()

Data normalization

In [5]:
normalized_df = sonar_data.copy()
for x in range(60):
    normalized_df[x] = MinMaxScaler().fit_transform(np.array(normalized_df[x]).reshape(-1,1))

normalized_df


In [6]:
normalized_df.describe()

Train test val split
80/10/10

In [7]:
X = sonar_data.drop(columns=60, axis=1)
y = sonar_data[60]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

In [9]:
print(X_train.shape)
print(y_train.value_counts())
print('******')
print(X_test.shape)
print(y_test.value_counts())
print('******')
print(X_val.shape)
print(y_val.value_counts())

Tensorflow

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from wandb.keras import WandbCallback

Vytvorenie modelu

In [ ]:
tf_model = Sequential()
tf_model.add(Dense(16, activation=tf.keras.activations.relu, input_dim=len(X_train.columns)))
tf_model.add(Dense(16, activation=tf.keras.activations.relu))
tf_model.add(Dense(16, activation=tf.keras.activations.relu))
tf_model.add(Dense(1, activation=tf.keras.activations.sigmoid))

In [ ]:
tf_model.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=Adam(), metrics=['accuracy'])

In [ ]:
tf_model.fit(
    X_train,
    y_train,
    batch_size=16,
    epochs=100,
    validation_data=(X_val,y_val)
)

In [ ]:
tf_model.save('./model')

In [ ]:
#pomocna funkcia
def predictEval(tf_model, XX, yy):
    # vykonanie predikcie
    y_pred = tf_model.predict(XX)
    # uprava outputu na boolean
    y_pred_bool = np.copy(y_pred)
    for x in y_pred_bool:
        x[0] = round(x[0])
    y_pred_bool

    #vratenie y a accuaracy
    return [y_pred, y_pred_bool, accuracy_score(y_pred_bool, yy)]

Eval

In [ ]:
train = predictEval(tf_model, X_train, y_train)
val = predictEval(tf_model, X_val, y_val)
test = predictEval(tf_model, X_test, y_test)

print('Accuracy score')
print(f'Train: {train[2]*100:.2f}%')
print(f'Val: {val[2]*100:.2f}%')
print(f'Test: {test[2]*100:.2f}%')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

In [ ]:
# Define the PyTorch model
class PyTorchModel(nn.Module):
    def __init__(self, input_dim):
        super(PyTorchModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

In [ ]:
# Create a custom dataset
class SonarDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_dataset = SonarDataset(X_train, y_train)
val_dataset = SonarDataset(X_val, y_val)
test_dataset = SonarDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [ ]:
# Instantiate the model
input_dim = len(X_train.columns)
model = PyTorchModel(input_dim)


In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())


In [ ]:
# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            y_val_pred = model(X_val_batch)
            val_loss = criterion(y_val_pred, y_val_batch)
    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

In [ ]:
def predict_eval(model, loader):
    with torch.no_grad():
        model.eval()
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            y_pred_bool = torch.round(y_pred)
            accuracy = accuracy_score(y_pred_bool, y_batch)
    return accuracy

In [ ]:
train_accuracy = predict_eval(model, train_loader)
val_accuracy = predict_eval(model, val_loader)
test_accuracy = predict_eval(model, test_loader)

In [ ]:
print('Accuracy score')
print(f'Train: {train_accuracy * 100:.2f}%')
print(f'Val: {val_accuracy * 100:.2f}%')
print(f'Test: {test_accuracy * 100:.2f}%')